In [ ]:
import numpy as np 
import pandas as pd 
import os
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import cv2
import matplotlib.pyplot as plt

In [ ]:
train_path =  "./dataset/train/"
classes = os.listdir(train_path)
classes

In [ ]:
# class_weight = [] 
# plt.subplots_adjust(hspace=0.5, wspace=0.5)

# for c in classes : 
#     class_path = os.path.join(train_path , c) 
#     img_path = os.path.join ( class_path , os.listdir(class_path)[0]) 
#     class_weight.append (len(os.listdir(class_path)) )
#     img = cv2.imread (img_path) 
#     plt.subplot(2 , 3 , classes.index(c)+1 ) 
#     plt.title (c)
#     plt.imshow(img)

In [ ]:
# plt.bar(classes, class_weight)    
# plt.xlabel("class")
# plt.ylabel("number of images" )


In [ ]:
from torchvision import transforms 
from torchvision.datasets import ImageFolder
from torch.utils.data import  DataLoader 

train_transform = transforms.Compose([
        transforms.Resize ( (150 , 150) ),
        #transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
test_transform =   transforms.Compose([
        transforms.Resize ( (150 , 150) ),        
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

In [ ]:
from torchvision import transforms, datasets
data_dir = 'dataset'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'

training_dataset = datasets.ImageFolder(train_dir, transform=train_transform )
validation_dataset = datasets.ImageFolder(valid_dir, transform=test_transform )
train = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
test = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
len(training_dataset)

In [ ]:
from torchvision import models 
model = models.vgg19(pretrained = True)

In [ ]:
for p in model.parameters() : 
    p.requires_grad = False 

In [ ]:
model.classifier = nn.Sequential(
      
  nn.Linear(in_features=25088, out_features=2048) ,
  nn.ReLU(),
  nn.Linear(in_features=2048, out_features=512) ,
  nn.ReLU(),
  nn.Dropout(p=0.6), 
    
  nn.Linear(in_features=512 , out_features=3),
  nn.LogSoftmax(dim=1)  
)


In [ ]:
model

In [ ]:
from torch import optim 
optimizer = optim.Adam( model.parameters() , lr = 0.00001)

In [ ]:
if torch.cuda.is_available() : 
    model = model.cuda()

In [ ]:
def accuracy (data) : 
    with torch.no_grad() : 
        model.eval()
        t = 0 
        c = 0
        for x , y in data :             
            preds = model(x.cuda()) 
            for i in range(preds.shape[0]) :  
                if torch.argmax(preds[i]) == y[i] :
                    c+=1              
                
                t +=1   
                
                
    return (c/t)      

In [ ]:
epochs = 5
step = 0
total_loss = 0 
train_acc = []
test_acc = []
for e in range (epochs) :
    l = 0
    for x , y in train : 
        model.zero_grad()
        output = model(x.cuda())
        loss = F.nll_loss(output, y.cuda())
        total_loss += loss.item()
        loss.backward() 
        optimizer.step()
        step += 1
        if  step % 20 == 0 :    
            train_acc.append( accuracy(train) )
            test_acc.append( accuracy(test) )
            print ( "step:" , step , " Loss:" , total_loss/300  ) 
            total_loss = 0 
            model.train()

       

In [ ]:
len(train_acc)

In [ ]:
plt.figure() 
plt.plot(train_acc , label = "train")
plt.plot(test_acc , label = "test" )
plt.legend()


In [ ]:
from PIL import Image

videoCapture = cv2.VideoCapture(r'./monkey.mp4')
fps = videoCapture.get(cv2.CAP_PROP_FPS)
size = (int(videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

ps = 25
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
label_ref = {1:"no Monkey", 0:"Monkey",2:"object"}
x = 0 
y = 0 
a =0 
z = 0
with torch.no_grad():
    success, frame = videoCapture.read()
    while success:
        frame_copy = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_copy = Image.fromarray(frame_copy)
        image_tensor = test_transform(frame_copy)
        image_tensor = image_tensor.unsqueeze(0) 
        test_input = image_tensor.to('cuda')
        outputs = model(test_input)
        _, predicted = torch.max(outputs, 1)
        probability =  F.softmax(outputs, dim=1)
        top_probability, top_class = probability.topk(1, dim=1)
        predicted = predicted.cpu().detach().numpy()
        predicted = predicted.tolist()[0]
        top_probability = top_probability.cpu().detach().numpy()
        top_probability = top_probability.tolist()[0][0]
        top_probability = '%.2f%%' % (top_probability * 100)
        print(top_probability)
        print(label_ref[predicted]) 
        z=z+1
        if(predicted == 1):
            x = x+1
        elif(predicted==0):
            y = y+1
        else:
            a = a+1
        frame = cv2.putText(frame, label_ref[predicted]+': '+top_probability, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
        success, frame = videoCapture.read()
print(" ")
print("non living")
print(a/z)
print("Human")
print(x/z)
print("monkey")
print(y/z)

In [ ]:
torch.save(model.state_dict(), './vgg19_v1_97_3O.pth')


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model =  model = models.vgg19(pretrained = True)
model.classifier = nn.Sequential(
      
  nn.Linear(in_features=25088, out_features=2048) ,
  nn.ReLU(),
  nn.Linear(in_features=2048, out_features=512) ,
  nn.ReLU(),
  nn.Dropout(p=0.6), 
    
  nn.Linear(in_features=512 , out_features=3),
  nn.LogSoftmax(dim=1)  
)
model = model.to(device)
model.load_state_dict(torch.load('./vgg19_v1_97_3O.pth'))
model.eval()
videoCapture = cv2.VideoCapture(r'./animals.mp4')
fps = videoCapture.get(cv2.CAP_PROP_FPS)
size = (int(videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

ps = 25
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
label_ref = {0:"Monkey", 1:"no Monkey",2:"object"}
x = 0 
y = 0 
a =0 
z = 0
with torch.no_grad():
    success, frame = videoCapture.read()
    while success:
        frame_copy = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_copy = Image.fromarray(frame_copy)
        image_tensor = test_transform(frame_copy)
        image_tensor = image_tensor.unsqueeze(0) 
        test_input = image_tensor.to(device)
        outputs = model(test_input)
        _, predicted = torch.max(outputs, 1)
        probability =  F.softmax(outputs, dim=1)
        top_probability, top_class = probability.topk(1, dim=1)
        predicted = predicted.cpu().detach().numpy()
        predicted = predicted.tolist()[0]
        top_probability = top_probability.cpu().detach().numpy()
        top_probability = top_probability.tolist()[0][0]
        top_probability = '%.2f%%' % (top_probability * 100)
        print(top_probability)
        print(label_ref[predicted]) 
        z=z+1
        if(predicted == 1):
            x = x+1
        elif(predicted==0):
            y = y+1
        else:
            a = a+ 1
        
        frame = cv2.putText(frame, label_ref[predicted]+': '+top_probability, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)
        success, frame = videoCapture.read()
print("monkey")
print(a/z)
print(x/z)
print(y/z)